<a href="https://colab.research.google.com/github/georgemoum-cylearn/gmlearn/blob/main/pcap_dataI4PS4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#εισαγωγή βιβλιοθηκών - πακέτων
import matplotlib.pyplot as plt
from  tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import os
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score,  f1_score, confusion_matrix
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [2]:
#φόρτωση του συνόλου δεδομένων
url = "https://raw.githubusercontent.com/jimtak80/AI_EKPA/main/Data/pcap_data.csv"
df = pd.read_csv(url)


In [3]:
#δημιουργία του συνόλου σε δύο κλάσεις Normal (0) και Attack (1)
X = df.drop("target", axis=1)
y = df["target"].apply(lambda x: 0 if x == 0.0 else 1)

In [4]:
#έλεγχος κλάσεων
print("Κατανομή κλάσεων y:")
print(y.value_counts())

Κατανομή κλάσεων y:
target
1    9965
0    5002
Name: count, dtype: int64


In [5]:
#διαχωρισμός κατηγορικών και αριθμητικών μεταβλητών
categorical_features = []
numeric_features = X.columns.difference(categorical_features).tolist()

In [6]:
#δημιουργία διοχέτευσης (αγωγού) προεπεξεργασίας με κωδικοποίηση μίας δέσμης (one-hot encoding) για κατηγορικές μεταβλητές
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [7]:
#τμηματοποίηση δεδομένων σε σύνολα εκαπίδευσης και δοκιμών
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print("Κατανομή κλάσεων y_train μετά τον διαχωρισμό:")
print(y_train.value_counts())

Κατανομή κλάσεων y_train μετά τον διαχωρισμό:
target
1    6975
0    3501
Name: count, dtype: int64


In [8]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Shape of X_train_processed:", X_train_processed.shape)
print("Shape of X_test_processed:", X_test_processed.shape)

Shape of X_train_processed: (10476, 25)
Shape of X_test_processed: (4491, 25)


In [9]:
#δημιουργία διοχέτευσης (αγωγού) SMOTE για τις αριθμητικές μεταβλητές μόνο
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [10]:
#καθορισμός κατωφλίου (threshold) για την διακοπή διακτυακής κίνησης
blocking_threshold = 0.9

In [11]:
#δημιουργία βρόχου συνεχούς - αυξητικής μάθησης
batch_size = 1000

# Shuffle X_train and y_train together to ensure mixed batches
shuffled_indices = np.random.permutation(len(X_train))
X_train_shuffled = X_train.iloc[shuffled_indices].reset_index(drop=True)
y_train_shuffled = y_train.iloc[shuffled_indices].reset_index(drop=True)

for epoch in range(1, 3):  #δυνατότητα αλλαγής των εποχών
    for i in range(0, len(X_train_shuffled), batch_size):
        X_batch = X_train_shuffled.iloc[i:i + batch_size]
        y_batch = y_train_shuffled.iloc[i:i + batch_size]

        # Ελέγξτε αν το y_batch έχει περισσότερες από μία κλάσεις
        if len(y_batch.unique()) < 2:
            print(f"Skipping batch {i} due to single class in y_batch.")
            continue

        #σταδιακή ενημέρωση του μοντέλου με κάθε ροή (batch) δεδομένων
        pipeline.fit(X_batch, y_batch)

        #περιοδική ενημέρωση του μοντέλου στο σύνολο δοκιμών
        #ΠΡΟΣΟΧΗ: Αυτή η συνθήκη ίσως χρειαστεί προσαρμογή για να ταιριάζει με τον αριθμό των batches
        #Για παράδειγμα, αν θέλετε να αξιολογήσετε μετά από κάθε batch, αφαιρέστε το `and i > 0`
        if i % batch_size == 0 and i > 0:
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

            #αποκλεισμός δικτυακής κυκλοφορίας εάν η προβλεπόμενη πιθανότητα υπερβαίνει το καθορισμένο όριο
            blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
            if len(blocked_indices) > 0:
                print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

            accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
            print(f"Epoch {epoch}, Iteration {i}, Test Accuracy: {accuracy}")

Blocking 795 malicious traffic instances.
Epoch 1, Iteration 1000, Test Accuracy: 0.5005566688933423
Blocking 1048 malicious traffic instances.
Epoch 1, Iteration 2000, Test Accuracy: 0.5493208639501225
Blocking 856 malicious traffic instances.
Epoch 1, Iteration 3000, Test Accuracy: 0.509686038744155
Blocking 857 malicious traffic instances.
Epoch 1, Iteration 4000, Test Accuracy: 0.5103540414161657
Blocking 690 malicious traffic instances.
Epoch 1, Iteration 5000, Test Accuracy: 0.47851258071698954
Blocking 740 malicious traffic instances.
Epoch 1, Iteration 6000, Test Accuracy: 0.48920062346916054
Blocking 907 malicious traffic instances.
Epoch 1, Iteration 7000, Test Accuracy: 0.5214874192830105
Blocking 717 malicious traffic instances.
Epoch 1, Iteration 8000, Test Accuracy: 0.48318859942106435
Blocking 711 malicious traffic instances.
Epoch 1, Iteration 9000, Test Accuracy: 0.48185259407704295
Blocking 672 malicious traffic instances.
Epoch 1, Iteration 10000, Test Accuracy: 0.47

In [12]:
#τελική αξιολόγηση του μοντέλου στο σύνολο δοκιμών
# Fit the pipeline on the full training data to ensure it's fitted for final evaluation
pipeline.fit(X_train, y_train)

y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
if len(blocked_indices) > 0:
    print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
classification_rep = classification_report(y_test, y_pred_proba > blocking_threshold)

#εκτύπωση τελικών αποτελεσμάτων
print(f"Final Test Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Blocking 1368 malicious traffic instances.
Final Test Accuracy: 0.6152304609218436
Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.96      0.63      1501
           1       0.96      0.44      0.60      2990

    accuracy                           0.62      4491
   macro avg       0.71      0.70      0.61      4491
weighted avg       0.79      0.62      0.61      4491

